# Artificial Neural Networks

In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm

In [2]:
df = pd.read_csv('dataset_NN.csv')
df

,att1,att2,att3,att4,att5,att6,class
0,1.0,0.0,0.0,133.150861,1.311693,1620.221779,1
1,1.0,0.0,0.0,126.724861,1.302745,1609.334822,1
2,1.0,0.0,0.0,131.173861,1.319031,1568.978435,1
3,1.0,0.0,0.0,129.478861,1.270878,1695.055281,1
4,1.0,0.0,0.0,127.262861,1.329637,1647.720235,1
...,...,...,...,...,...,...,...
1995,1.0,1.0,1.0,157.498861,1.655794,5326.025889,10
1996,1.0,1.0,1.0,152.404861,1.620345,5243.267754,10
1997,1.0,1.0,1.0,134.672861,1.541987,3766.763222,10
1998,1.0,1.0,1.0,142.926861,1.426381,4118.327320,10


In [3]:
df['class'].value_counts()

10    200
9     200
8     200
7     200
6     200
5     200
4     200
3     200
2     200
1     200
Name: class, dtype: int64

In [4]:
df.values.shape

(2000, 7)

In [5]:
X = df.values[:,:-1]
y = df.values[:, -1]

In [9]:
layer_dimensions = [7,7,8,10]

In [22]:
def initialize(layer_dimensions):
    np.random.seed(2)
    
    params = {}
    
    for l in range(1,len(layer_dimensions)):
        params['W' + str(l)] = np.random.randn(layer_dimensions[l], layer_dimensions[l-1])*0.01
        
        params['b' + str(l)] = np.random.randn(layer_dimensions[l], 1)*0.01
    
    return params
    

In [30]:
params = initialize(layer_dimensions)
for l in range(1,len(layer_dimensions)):
    print(params['W'+str(l)].shape)

(7, 7)
(8, 7)
(10, 8)
